In [1]:
%matplotlib inline

ModuleNotFoundError: No module named 'matplotlib'

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
%matplotlib inline
abaloneDF = pd.read_csv('abalone.csv', names=['Sex', 'Length', 'Diameter', 'Height',
                                              'Whole Weight', 'Shucked Weight',
                                              'Viscera Weight', 'Shell Weight',
                                              'Rings'])
abaloneDF['Male'] = (abaloneDF['Sex'] == 'M').astype(int)
abaloneDF['Female'] = (abaloneDF['Sex'] == 'F').astype(int)
abaloneDF['Infant'] = (abaloneDF['Sex'] == 'I').astype(int)
abaloneDF = abaloneDF[abaloneDF['Height'] > 0]

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
#Y-Yhat plot, as before
def plot_yyhat(ytest,ypred):
    r2 = r2_score(ytest, ypred )
    mae = mean_absolute_error(ytest, ypred)
    absmin = min([ytest.min(),ypred.min()])
    absmax = max([ytest.max(),ypred.max()])
    ax = plt.axes()
    ax.scatter(ytest,ypred)
    ax.set_title('Y vs. YHat')
    ax.axis([absmin, absmax, absmin, absmax])
    ax.plot([absmin, absmax], [absmin, absmax],c="k")
    ax.set_ylabel('Predicted Rings') 
    ax.set_xlabel('Actual Rings')
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    textStr = '$MAE=%.3f$\n$R2=%.3f$' % (mae, r2)
    ax.text(0.05, 0.95, textStr, transform=ax.transAxes, fontsize=14,
               verticalalignment='top', bbox=props);
#Cross-validation, as before
train, test = train_test_split(abaloneDF,train_size=0.7)
xtrain = train.drop(['Rings','Sex'],axis=1)
ytrain = train['Rings']
xtest = test.drop(['Rings','Sex'],axis=1)
ytest = test['Rings']
#Scale the data
stScaler = StandardScaler()  
stScaler.fit(xtrain)  
xtrain2 = stScaler.transform(xtrain)  
xtest2 = stScaler.transform(xtest)  
#Construct a multi-layer perceptron 
mlp = MLPRegressor(hidden_layer_sizes = [20,5],
                   alpha=0.001,
                   activation = 'logistic',
                   learning_rate_init = 0.001,
                   max_iter = 1000)
mlp.fit(xtrain2,ytrain)
ypred = mlp.predict(xtest2)
yresid = ytest - ypred
plot_yyhat(ytest,ypred)

In [3]:
#Obtain MAE from modifying the activation function
def activationResults(func):
    mlp = MLPRegressor(hidden_layer_sizes = [10,10],
                       alpha=0.00001,
                       activation = func,
                       learning_rate_init = 0.001,
                       max_iter = 2000)
    mlp.fit(xtrain2,ytrain)
    ypred = mlp.predict(xtest2)
    return mean_absolute_error(ytest, ypred)
#Run the possibilities in a loop
dictResults = {}
for func in ['identity','logistic','tanh','relu']:
    dictResults[func] = activationResults(func)
#Print the results
dictResults

NameError: name 'MLPRegressor' is not defined

In [4]:
#Obtain MAE from modifying alpha
def alphaResults(alpha):
    mlp = MLPRegressor(hidden_layer_sizes = [10,10],
                       alpha=alpha,
                       activation = 'logistic',
                       learning_rate_init = 0.001,
                       max_iter = 2000)
    mlp.fit(xtrain2,ytrain)
    ypred = mlp.predict(xtest2)
    return mean_absolute_error(ytest, ypred)
#Run the possibilities in a loop
results2 = []
alphas = np.logspace(-6, 0, num=35)
for alpha in alphas:
    results2.append(alphaResults(alpha))
#Plot the results
plt.plot(alphas,results2)
plt.xscale('log')
plt.xlabel('Alpha')
plt.ylabel('MAE');

NameError: name 'MLPRegressor' is not defined

In [5]:
#Obtain MAE from modifying the hidden layers
def hlayerResults(hlayer):
    mlp = MLPRegressor(hidden_layer_sizes = hlayer,
                       alpha= 0.01,
                       activation = 'logistic',
                       learning_rate_init = 0.001,
                       max_iter = 2000)
    mlp.fit(xtrain2,ytrain)
    ypred = mlp.predict(xtest2)
    return mean_absolute_error(ytest, ypred)
#Let's try some "typical" values, with 2 layers, all combinations
#of 5-spaced intervals
hlayers = [[x,y] for x in range(5,21,5) for y in range(5,21,5)]
#Now let's add a few "weirder" ones to see what happens:
hlayers.extend([[1,10],[10,1],[2,2],[10,2,10],[10,5,10],
                [20,5,20],[20,2,20],[10],[20]])
dfResults = pd.DataFrame(columns=['MAE'])
for hlayer in hlayers:
    dfResults.loc[str(hlayer)] = {'MAE' : hlayerResults(hlayer)}
#Print the results
dfResults.sort_values('MAE')

NameError: name 'MLPRegressor' is not defined

In [ ]:
#Obtain MAE from modifying learning rate
def lrResults(lr):
    mlp = MLPRegressor(hidden_layer_sizes = [20,5],
                       alpha=0.01,
                       activation = 'logistic',
                       learning_rate_init = lr,
                       max_iter = 4000)
    mlp.fit(xtrain2,ytrain)
    ypred = mlp.predict(xtest2)
    return mean_absolute_error(ytest, ypred)
#Run the possibilities in a loop
results4 = []
lrs = np.logspace(-4, -1, num=20)
for lr in lrs:
    results4.append(lrResults(lr))
#Plot the results
plt.plot(lrs,results4)
plt.xscale('log')
plt.xlabel('Learning Rate')
plt.ylabel('MAE');